# Import Pandas and Plotly (offline mode)

In [1]:
import pandas as pd

import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

pd.set_option("display.max_columns", 0) # don't truncate columns
pd.set_option("display.max_colwidth", 0) # don't truncate column values

init_notebook_mode(connected=True)

# Read in Caliper Data 

Caliper key-value data formatted in JSON can be read directly with Pandas.

This particular dataset was generated using the configuration file [configs/data_centric_PEBS.config](https://github.com/LLNL/caliper-examples/blob/master/configs/data_centric_PEBS.conf)

Now, create a Pandas dataframe from the JSON data using `read_json` as follows:

In [2]:
df = pd.read_json('../datasets/lulesh_latency_analysis.json')
df[['function','libpfm.operation','libpfm.memory_level','libpfm.hit_type','libpfm.weight']].head()

,function,libpfm.operation,libpfm.memory_level,libpfm.hit_type,libpfm.weight
0,main/LagrangeLeapFrog/LagrangeNodal/CalcForceForNodes,NaN,NaN,NaN,NaN
1,main/LagrangeLeapFrog/LagrangeNodal/CalcForceForNodes/CalcVolumeForceForElems/IntegrateStressForElems,Load,L1,Hit,15.0
2,main/LagrangeLeapFrog/LagrangeNodal/CalcForceForNodes/CalcVolumeForceForElems/CalcHourglassControlForElems,Load,L1,Hit,319.0
3,main/LagrangeLeapFrog/LagrangeNodal/CalcForceForNodes/CalcVolumeForceForElems/CalcHourglassControlForElems/CalcFBHourglassForceForElems,Load,L1,Hit,12.0
4,main/LagrangeLeapFrog/LagrangeNodal/CalcForceForNodes/CalcVolumeForceForElems/CalcHourglassControlForElems/CalcFBHourglassForceForElems,Load,L2,Hit,16.0


In [3]:
df.columns

Index(['alloc.index#libpfm.addr', 'alloc.label#libpfm.addr',
       'alloc.total_size', 'alloc.uid', 'alloc.uid#libpfm.addr',
       'cali.caliper.version', 'cali.event.begin', 'cali.event.end',
       'cali.event.set', 'callpath.address', 'event.begin#function',
       'event.begin#iteration#lulesh.cycle', 'event.begin#loop',
       'event.begin#lulesh.region', 'event.end#function',
       'event.end#iteration#lulesh.cycle', 'event.end#loop',
       'event.end#lulesh.region', 'event.set#pthread.id', 'function',
       'instruction.op#callpath.address',
       'instruction.read_size#callpath.address',
       'instruction.write_size#callpath.address', 'iteration#lulesh.cycle',
       'libpfm.addr',
       'libpfm.counter.MEM_TRANS_RETIRED:LATENCY_ABOVE_THRESHOLD',
       'libpfm.cpu', 'libpfm.data_src', 'libpfm.event_sample_name',
       'libpfm.hit_type', 'libpfm.memory_level', 'libpfm.operation',
       'libpfm.snoop', 'libpfm.tid', 'libpfm.time', 'libpfm.tlb',
       'libpfm.weight',

We clean things up a bit by sorting by time and deriving the `base_function` column:

In [4]:
import os
df = df.sort_values('libpfm.time')
df['base_function'] = df['function'].transform(lambda s: os.path.basename(str(s)))

# Data Aggregations

## Data-Centric Profiles

First, we plot the sum of latencies per allocation:

In [5]:
df_data_centric = df[['alloc.label#libpfm.addr', 'libpfm.weight']].groupby('alloc.label#libpfm.addr').sum()
data = [
    go.Bar(
        name='Allocation Quantity per Function',
        x=df_data_centric.index, 
        y=df_data_centric['libpfm.weight']
    )
]
iplot(data)

Looks like `fy`, `fz`, `nodelist`, `z`, and `zd`  stand out, but they may have just been accessed more frequently, rather than inefficiently.

Lets do the same plot, but average the latencies instead of sum them:

In [6]:
df_data_centric = df[['alloc.label#libpfm.addr', 'libpfm.weight']].groupby('alloc.label#libpfm.addr').mean()
data = [
    go.Bar(
        name='Allocation Quantity per Function',
        x=df_data_centric.index, 
        y=df_data_centric['libpfm.weight']
    )
]
iplot(data)

Now it looks like `dzz`, `dxx`, stand out. Lets take a look at that data:

In [11]:
df_dzz = df[df['alloc.label#libpfm.addr'].isin(['dzz', 'dxx', 'delx_xi', 'delx_eta'])]

df_dzz[['function',
        'alloc.label#libpfm.addr',
        'libpfm.operation',
        'libpfm.memory_level', 
        'libpfm.hit_type', 
        'libpfm.weight',
        'libpfm.snoop']].sort_values('libpfm.weight', ascending=False).head(20)

,function,alloc.label#libpfm.addr,libpfm.operation,libpfm.memory_level,libpfm.hit_type,libpfm.weight,libpfm.snoop
10796,main/LagrangeLeapFrog/LagrangeElements/CalcLagrangeElements,dzz,Load,LFB,Hit,1659.0,None
5122,main/LagrangeLeapFrog/LagrangeElements/CalcQForElems/CalcMonotonicQRegionForElems,delx_xi,Load,Remote RAM 1 Hop,Hit,1499.0,None
10357,main/LagrangeLeapFrog/LagrangeElements/CalcQForElems/CalcMonotonicQRegionForElems,delx_eta,Load,Remote RAM 1 Hop,Hit,1383.0,None
7819,main/LagrangeLeapFrog/LagrangeElements/CalcQForElems/CalcMonotonicQRegionForElems,delx_xi,Load,Remote Cache 1 Hops,Hit,1362.0,Hit Modified
8210,main/LagrangeLeapFrog/LagrangeElements/CalcQForElems/CalcMonotonicQRegionForElems,delx_xi,Load,Remote RAM 1 Hop,Hit,1333.0,None
13645,main/LagrangeLeapFrog/LagrangeElements/CalcLagrangeElements,dxx,Load,LFB,Hit,1108.0,None
5964,main/LagrangeLeapFrog/LagrangeElements/CalcQForElems/CalcMonotonicQRegionForElems,delx_xi,Load,Remote RAM 1 Hop,Hit,898.0,None
4097,main/LagrangeLeapFrog/LagrangeElements/CalcQForElems/CalcMonotonicQRegionForElems,delx_eta,Load,Remote RAM 1 Hop,Hit,814.0,None
14634,main/LagrangeLeapFrog/LagrangeElements/CalcQForElems/CalcMonotonicQRegionForElems,delx_eta,Load,Local RAM,Hit,597.0,None
4946,main/LagrangeLeapFrog/LagrangeElements/CalcLagrangeElements,dxx,Load,LFB,Hit,556.0,None


## Call Tree Aggregations of Data-Centric Profiles

We are first and foremost interested in functions that contribute most to execution time.

We can combine our allocation label information with call tree aggregation in Hatchet to narrow down accesses inside of long-executing functions.

In [8]:
from cali_analysis import hatchet

pd.options.mode.chained_assignment = None  # default='warn'

# Derive a tuple from the function path by splitting on '/'
df_callpaths = df.dropna(subset=['function'])
df_callpaths['function'] = df_callpaths['function'].transform(lambda l: tuple(l.split('/')))
df_callpaths['alloc.label'] = df_callpaths['alloc.label#libpfm.addr'].transform(lambda l: set([l]) )

# Construct a call tree, aggregating allocation sizes at each node
mrt = hatchet.MultiRootTree.from_samples(df_callpaths, 'function', {
    'time.inclusive.duration' : 'max',
    'libpfm.weight' : 'sum',
    'alloc.label' : lambda s: set.union(*s)
})
mrt.df_nodes[['depth#function', 'function', 'time.inclusive.duration', 'libpfm.weight']].sort_values(['depth#function', 'time.inclusive.duration'], ascending=[True, False])

,depth#function,function,time.inclusive.duration,libpfm.weight
0,1,"(main,)",7356803.0,1498571.0
0,2,"(main, LagrangeLeapFrog)",426843.0,1494949.0
1,2,"(main, TimeIncrement)",779.0,0.0
4,3,"(main, LagrangeLeapFrog, LagrangeNodal)",268656.0,1133676.0
5,3,"(main, LagrangeLeapFrog, LagrangeElements)",162764.0,349359.0
3,3,"(main, LagrangeLeapFrog, CalcTimeConstraintsForElems)",71942.0,11914.0
5,4,"(main, LagrangeLeapFrog, LagrangeNodal, CalcForceForNodes)",247724.0,1081581.0
4,4,"(main, LagrangeLeapFrog, LagrangeElements, ApplyMaterialPropertiesForElems)",100833.0,202305.0
7,4,"(main, LagrangeLeapFrog, LagrangeElements, CalcQForElems)",56844.0,89149.0
3,4,"(main, LagrangeLeapFrog, LagrangeElements, CalcLagrangeElements)",29421.0,57344.0


In [9]:
# Get the first and only tree, rooted at 'main'
tree = hatchet.Tree(mrt.roots[0])
hp = tree.hot_path('time.inclusive.duration')
hp[['time.inclusive.duration','libpfm.weight','alloc.label','function']]

,time.inclusive.duration,libpfm.weight,alloc.label,function
0,7356803.0,1498571.0,"{nan, dyy, volo, lzetam, delv, elemBC, delx_xi, lxim, fy, p, ss, nodalMass, arealg, nodelist, qq, ql, delv_eta, vdov, yd, xdd, dxx, e, q, dzz, ydd, v, z, y, delx_zeta, lzetap, delv_zeta, fz, letam, lxip, xd, zd, zdd, delv_xi, fx, delx_eta, x, letap, elemMass}","(main,)"
0,426843.0,1494949.0,"{nan, dyy, volo, lzetam, delv, elemBC, delx_xi, lxim, fy, p, ss, nodalMass, arealg, nodelist, qq, ql, delv_eta, vdov, yd, xdd, dxx, e, q, dzz, ydd, v, z, y, delx_zeta, lzetap, delv_zeta, fz, letam, lxip, xd, zd, zdd, delv_xi, fx, delx_eta, x, letap, elemMass}","(main, LagrangeLeapFrog)"
4,268656.0,1133676.0,"{nan, ydd, volo, z, v, delv, y, fy, p, ss, fz, nodalMass, xd, zd, zdd, nodelist, qq, fx, x, yd, xdd, elemMass, q}","(main, LagrangeLeapFrog, LagrangeNodal)"
5,247724.0,1081581.0,"{nan, volo, z, v, delv, y, fy, p, ss, fz, nodalMass, xd, zd, nodelist, qq, fx, x, yd, elemMass, q}","(main, LagrangeLeapFrog, LagrangeNodal, CalcForceForNodes)"
8,247001.0,1081379.0,"{nan, volo, z, v, delv, y, fy, p, ss, fz, nodalMass, xd, zd, nodelist, qq, fx, x, yd, elemMass, q}","(main, LagrangeLeapFrog, LagrangeNodal, CalcForceForNodes, CalcVolumeForceForElems)"
5,186653.0,712526.0,"{nan, v, z, volo, delv, y, fy, ss, fz, zd, xd, nodelist, qq, fx, x, yd, elemMass, q}","(main, LagrangeLeapFrog, LagrangeNodal, CalcForceForNodes, CalcVolumeForceForElems, CalcHourglassControlForElems)"
23,93395.0,361098.0,"{nan, zd, xd, qq, nodelist, fx, fy, yd, elemMass, ss, q, fz}","(main, LagrangeLeapFrog, LagrangeNodal, CalcForceForNodes, CalcVolumeForceForElems, CalcHourglassControlForElems, CalcFBHourglassForceForElems)"
